In [1]:
! pip install pettingzoo
! pip install gymnasium
! pip install stable_baselines3
! pip install ta
! pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 816.1/816.1 KB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 KB 33.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.8/171.8 KB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 28.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gym: filename=gym-0.21.0-py3-none-any.whl size=1616820 sha256=49a55f6f7c8c27f3b9c5aca88132f4debb8df1c51a442d0c19f005f25e85f36c
  Stored in directory: /root/.cache/pip/wheels/27/6d/b3/a3a6e10704795c9b9000f1ab2dc480dfe7bed42f5972806e73
Successfully built gym
  Attempting uninstall: importlib-metadata
    Found existing i

# Imports

In [74]:
import pettingzoo
from pettingzoo import AECEnv, ParallelEnv
from pettingzoo.utils import agent_selector, wrappers
import gymnasium
import gym
from stable_baselines3 import A2C, DDPG, DQN, PPO
from ta import add_all_ta_features
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import functools

# Data

## Tech Indicators

In [75]:
def add_ta(df):
  ta_df = add_all_ta_features(df, open="Open", high="High", low="Low", close="Close", volume="Volume")
  print(ta_df.columns)
  ta_df = ta_df[["Open", "High", "Low", "Close", "Volume", "Adj Close", "volume_obv",
                                "volume_adi", "trend_adx", "momentum_ao", "trend_macd", "momentum_rsi", 
                                "momentum_stoch"]]
  ta_df = ta_df.fillna(ta_df.mean())
  return ta_df

## Stocks

1. Apple Inc. (AAPL)
2. Microsoft Corp. (MSFT)
3. Amazon.com, Inc. ( AMZN)
4. Tesla, Inc. (TSLA)
5. Nvidia Corp. (NVDA)

In [76]:
# interval = 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
# prepost = T/F

aapl_df = yf.download("AAPL", start="2018-01-01", end="2022-12-31", keepna=True)
aapl_df = aapl_df.fillna(aapl_df.mean())
aapl_df = add_ta(aapl_df)

# msft_df = yf.download("MSFT", start="2022-12-01", end="2022-12-31", keepna=True)
# amzn_df = yf.download("AMZN", start="2022-12-01", end="2022-12-31", keepna=True)
# tsla_df = yf.download("TSLA", start="2022-12-01", end="2022-12-31", keepna=True)
# nvda_df = yf.download("NVDA", start="2022-12-01", end="2022-12-31", keepna=True)

caas_df = yf.download("CAAS", start="2018-01-01", end="2022-12-31", keepna=True)
caas_df = caas_df.fillna(caas_df.mean())
caas_df = add_ta(caas_df)

# asc_df = yf.download("ASC", start="2022-12-01", end="2022-12-31", keepna=True)

[*********************100%***********************]  1 of 1 completed


/usr/local/lib/python3.8/dist-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
/usr/local/lib/python3.8/dist-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)


Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'volume_adi',
       'volume_obv', 'volume_cmf', 'volume_fi', 'volume_em', 'volume_sma_em',
       'volume_vpt', 'volume_vwap', 'volume_mfi', 'volume_nvi',
       'volatility_bbm', 'volatility_bbh', 'volatility_bbl', 'volatility_bbw',
       'volatility_bbp', 'volatility_bbhi', 'volatility_bbli',
       'volatility_kcc', 'volatility_kch', 'volatility_kcl', 'volatility_kcw',
       'volatility_kcp', 'volatility_kchi', 'volatility_kcli',
       'volatility_dcl', 'volatility_dch', 'volatility_dcm', 'volatility_dcw',
       'volatility_dcp', 'volatility_atr', 'volatility_ui', 'trend_macd',
       'trend_macd_signal', 'trend_macd_diff', 'trend_sma_fast',
       'trend_sma_slow', 'trend_ema_fast', 'trend_ema_slow',
       'trend_vortex_ind_pos', 'trend_vortex_ind_neg', 'trend_vortex_ind_diff',
       'trend_trix', 'trend_mass_index', 'trend_dpo', 'trend_kst',
       'trend_kst_sig', 'trend_kst_diff', 'trend_ichimoku_conv',
       

In [77]:
def add_cov(df):
  df = df.reset_index()

  cov_list = []
  return_list = []

  # look back is one year
  lookback=252
  for i in range(lookback,len(df.index.unique())):
    data_lookback = df.iloc[i-lookback:i,:]
    price_lookback=data_lookback.pivot_table(index = 'Date', values = 'Close')
    return_lookback = price_lookback.pct_change().dropna()
    return_list.append(return_lookback)

    covs = return_lookback.cov().values 
    cov_list.append(covs)


  df_cov = pd.DataFrame({'Date':df["Date"].unique()[lookback:],'cov_list':cov_list,'return_list':return_list})
  df = df.merge(df_cov, on='Date')
  df = df.sort_values(['Date']).reset_index(drop=True)
  return df

In [78]:
aapl_df = add_cov(aapl_df)
caas_df = add_cov(caas_df)

In [79]:
new_aapl_df = aapl_df.copy()
new_aapl_df["tic"] = "AAPL"
new_caas_df = caas_df.copy()
new_caas_df["tic"] = "CAAS"
mixed_df = pd.concat([new_aapl_df, new_caas_df])

# Agent

In [80]:
AGENT_STR_TO_OBJECT = {
    "A2C": A2C,
    "DDPG": DDPG,
    "PPO": PPO,
}
policy = "MlpPolicy"

# AEC Environment

## Env

In [81]:
def marl_env(render_mode=None, **kwargs):
  internal_render_mode = render_mode if render_mode != "ansi" else "human"
  env = raw_env(render_mode=internal_render_mode, **kwargs)
  # This wrapper is only for environments which print results to the terminal
  if render_mode == "ansi":
      env = wrappers.CaptureStdoutWrapper(env)
  # this wrapper helps error handling for discrete action spaces
  env = wrappers.AssertOutOfBoundsWrapper(env)
  # Provides a wide vareity of helpful user errors
  # Strongly recommended
  env = wrappers.OrderEnforcingWrapper(env)
  return env

class raw_env(AECEnv):
  metadata = {"render_modes": ["human"], "name": "marlpm_v1"}

  def __init__(
      self, 
      df,
      stock_dim,
      hmax,
      initial_amount,
      transaction_cost_pct,
      reward_scaling,
      state_space,
      action_space,
      tech_indicator_list,
      turbulence_threshold=None,
      lookback=252,
      day=0,
      render_mode=None,
  ):

    # attributes
    self.lookback=lookback
    self.df = df
    self.stock_dim = stock_dim
    self.hmax = hmax
    self.initial_amount = initial_amount
    self.transaction_cost_pct =transaction_cost_pct
    self.reward_scaling = reward_scaling
    self.state_space = state_space
    self.action_space = action_space
    self.tech_indicator_list = tech_indicator_list

    # agents
    self.possible_agents = ["A2C", "DDPG", "PPO"]

    # spaces
    # self.action_space = {agent: gym.spaces.Box(low = -1, high = 1, shape = (self.action_space,)) for agent in self.possible_agents}
    # self.observation_space = {
    #     agent: gym.spaces.Box(low=0, high=np.inf, shape = (1+len(self.tech_indicator_list), self.state_space)) for agent in self.possible_agents
    # }
    # to fix?
    self.action_space = gym.spaces.Box(low = -1, high = 1, shape = (self.action_space,))
    self.observation_space = gym.spaces.Box(low=0, high=np.inf, shape = (1+len(self.tech_indicator_list), self.state_space))

    # agents setup
    self.agent_name_mapping = {
        agent: AGENT_STR_TO_OBJECT[agent](policy, self) for agent in self.possible_agents
    }
    self.training_agent = None
    self.day = {
        agent: day for agent in self.possible_agents
    }

    # data
    self.data = {
        agent: self.df.loc[self.day[agent],:] for agent in self.possible_agents
    }
    self.covs = {
        agent: [[x[0][0] for x in self.data[agent]['cov_list']]] for agent in self.possible_agents
    }
    self.state = {
        agent: np.append(np.array(self.covs[agent]), [self.data[agent][tech].values.tolist() for tech in self.tech_indicator_list ], axis=0) for agent in self.possible_agents
    }
    # self.data = self.df.loc[self.day["A2C"],:] # to fix
    # self.covs = [[x[0][0] for x in self.data['cov_list']]]
    # self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
    self.terminal = False     
    self.turbulence_threshold = turbulence_threshold   

    # memory
    self.portfolio_value = {
        agent: self.initial_amount for agent in self.possible_agents
    }
    self.asset_memory = {
        agent: [self.initial_amount] for agent in self.possible_agents
    }
    self.portfolio_return_memory = {
        agent: [0] for agent in self.possible_agents
    }
    self.actions_memory = {
        agent: [[1/self.stock_dim]*self.stock_dim] for agent in self.possible_agents
    }
    self.date_memory = {
        agent: [self.data[agent]["Date"].unique()[0]] for agent in self.possible_agents
    }

    # observations (todo)
    self.observations = {
        agent: None for agent in self.possible_agents
    }

    # render mode
    self.render_mode = render_mode

  # @functools.lru_cache(maxsize=None)
  # def observation_space(self, agent):
  #   # gymnasium spaces are defined and documented here: https://gymnasium.farama.org/api/spaces/
  #   return gym.Box(low=0, high=np.inf, shape = (1+len(self.tech_indicator_list), self.state_space))

  # @functools.lru_cache(maxsize=None)
  # def action_space(self, agent):
  #   return gym.Box(low = -1, high = 1, shape = (self.action_space,))
  
  def step(self, actions):
    # print(self.day)
    # print(actions)

    # termination
    # to fix
    self.terminal = self.day[self.training_agent] >= len(self.df.index.unique())/self.stock_dim-1
    # self.terminal = {
    #     agent: self.day >= len(self.df.index.unique())/self.stock_dim-1 for agent in self.possible_agents
    # }

    if self.terminal:
      df = pd.DataFrame(self.portfolio_return_memory[self.training_agent])
      df.columns = ['daily_return']
      plt.plot(df.daily_return.cumsum(),'r')
      # plt.savefig('results/cumulative_reward.png')
      plt.close()

      plt.plot(self.portfolio_return_memory[self.training_agent],'r')
      # plt.savefig('results/rewards.png')
      plt.close()

      print("=================================")
      print("begin_total_asset:{}".format(self.asset_memory[self.training_agent][0]))           
      print("end_total_asset:{}".format(self.portfolio_value[self.training_agent]))

      df_daily_return = pd.DataFrame(self.portfolio_return_memory[self.training_agent])
      df_daily_return.columns = ['daily_return']
      if df_daily_return['daily_return'].std() !=0:
        sharpe = (252**0.5)*df_daily_return['daily_return'].mean()/ \
                  df_daily_return['daily_return'].std()
        print("Sharpe: ",sharpe)
      print("=================================")

      return self.state[self.training_agent], self.reward[self.training_agent], self.terminal, {}

    else:
      for agent in self.possible_agents:
        # normalisation?
        weights = self.softmax_normalization(actions) 
        self.actions_memory[agent].append(weights)
        last_day_memory = self.data[agent]

        #load next state
        self.day[agent] += 1
        self.data[agent] = self.df.loc[self.day[agent],:]
        self.covs[agent] = [[x[0][0] for x in self.data[agent]['cov_list']]]
        self.state[agent] =  np.append(np.array(self.covs[agent]), [self.data[agent][tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        # calculate portfolio return
        # individual stocks' return * weight
        portfolio_return = sum(((self.data[agent]["Close"].values / last_day_memory["Close"].values)-1)*weights)
        # update portfolio value
        new_portfolio_value = self.portfolio_value[agent]*(1+portfolio_return)
        self.portfolio_value[agent] = new_portfolio_value

        # save into memory
        self.portfolio_return_memory[agent].append(portfolio_return)
        self.date_memory[agent].append(self.data[agent]["Date"].unique()[0])            
        self.asset_memory[agent].append(new_portfolio_value)

        # the reward is the new portfolio value or end portfolo value
        self.reward[agent] = new_portfolio_value 
        #self.reward = self.reward*self.reward_scaling
        # self.reward *= (self.reward / pv)
      
      for agent, pv in self.reward.items():
        if agent == self.training_agent:
          continue
        self.reward[self.training_agent] *= (self.reward[self.training_agent] / pv)

    return self.state[agent], self.reward[self.training_agent], self.terminal, {}

  def observe(self, agent):
    # to fix
    return self.observations[agent]

  def reset(self, seed=None, return_info=False, options=None):
    print("reset")
    
    # agents
    self.agents = self.possible_agents[:]
    # self.agent_selector = agent_selector(self.agents)
    # self.curr_agent = self.agent_selector.next()

    # attributes
    self.day = {
        agent: 0 for agent in self.possible_agents
    }
    self.data = {
        agent: self.df.loc[self.day[agent],:] for agent in self.possible_agents
    }
    self.covs = {
        agent: [[x[0][0] for x in self.data[agent]['cov_list']]] for agent in self.possible_agents
    }
    self.state = {
        agent: np.append(np.array(self.covs[agent]), [self.data[agent][tech].values.tolist() for tech in self.tech_indicator_list ], axis=0) for agent in self.possible_agents
    }

    # memory
    self.portfolio_value = {
        agent: self.initial_amount for agent in self.possible_agents
    }
    self.asset_memory = {
        agent: [self.initial_amount] for agent in self.possible_agents
    }
    self.portfolio_return_memory = {
        agent: [0] for agent in self.possible_agents
    }
    self.actions_memory = {
        agent: [[1/self.stock_dim]*self.stock_dim] for agent in self.possible_agents
    }
    self.date_memory = {
        agent: [self.data[agent]["Date"].unique()[0]] for agent in self.possible_agents
    }

    # rewards
    self.reward = {
        agent: None for agent in self.possible_agents
    }
    self.observations = {
        agent: None for agent in self.possible_agents
    }
    
    # misc
    self.terminal = False 
    #self.cost = 0
    #self.trades = 0

    return self.state[self.training_agent]

  def render(self):
    return self.state[self.training_agent]

  def seed(self, seed=None):
    self.np_random, seed = seeding.np_random(seed)
    return [seed]

  def close(self):
    pass
  
  def softmax_normalization(self, actions):
    numerator = np.exp(actions)
    denominator = np.sum(np.exp(actions))
    softmax_output = numerator/denominator
    return softmax_output

  def set_training_agent(self, agent):
    print(agent)
    self.training_agent = agent

  def learn(self, total_timesteps=1000):
    for agent in self.possible_agents:
      self.set_training_agent(agent)
      self.agent_name_mapping[agent] = self.agent_name_mapping[agent].learn(total_timesteps=total_timesteps)

## Demo

In [82]:
stock_dimension = len(mixed_df["tic"].unique())
state_space = 2

env_kwargs = {
  "hmax": 100, 
  "initial_amount": 1000000, 
  "transaction_cost_pct": 0.001, 
  "state_space": state_space, 
  "stock_dim": stock_dimension, 
  "tech_indicator_list": [
    "volume_obv",
    "volume_adi", 
    "trend_adx", 
    "momentum_ao", 
    "trend_macd", 
    "momentum_rsi", 
    "momentum_stoch"
  ], 
  "action_space": stock_dimension, 
  "reward_scaling": 1e-4,
    
}

In [83]:
test_env = raw_env(df=mixed_df, **env_kwargs)

## Train

In [ ]:
test_env.set_training_agent("A2C")
test_env.learn(
    total_timesteps=1000,
)

A2C
A2C
reset
begin_total_asset:1000000
end_total_asset:5362344.637432834
Sharpe:  1.276287990876499
reset
DDPG
reset
begin_total_asset:1000000
end_total_asset:3604241.3692432614
Sharpe:  1.5860838968062432
reset
begin_total_asset:1000000
end_total_asset:3204377.9254798056
Sharpe:  1.4957033991275601
reset
PPO
reset
begin_total_asset:1000000
end_total_asset:3714648.0341599225
Sharpe:  0.9976891093099292
reset


# Parallel Environment